# Quantdle Python Client - Comprehensive Examples

This notebook demonstrates how to use the Quantdle Python client to download historical financial market data.

## Features Demonstrated

- **Basic data downloading** - Simple data retrieval for analysis
- **Multiple symbols and timeframes** - Efficient batch downloads  
- **Pandas vs Polars DataFrames** - Choose the right tool for your data size
- **Large date range handling** - Automatic chunking for big datasets
- **Advanced configuration** - Customizing downloads for optimal performance
- **Symbol discovery** - Finding available data and getting symbol information
- **Error handling** - Robust code for production use
- **Technical analysis** - Real-world data analysis examples

## Prerequisites

```bash
# Install the quantdle package
pip install quantdle

# Optional: For polars support
pip install quantdle[polars]
```

## Setup

Set your API credentials as environment variables for security:

```bash
export QDL_API_KEY='your-api-key'
export QDL_API_KEY_ID='your-api-key-id'
```


In [1]:
# Import required libraries
import os
from datetime import datetime, timedelta
import quantdle as qdl
import pandas as pd

# Setup function
def setup_client():
    """
    Initialize the Quantdle client with your API credentials.
    
    For security, it's recommended to store credentials as environment variables:
    - QDL_API_KEY: Your Quantdle API key
    - QDL_API_KEY_ID: Your Quantdle API key ID
    """
    
    # Option 1: Use environment variables (recommended)
    api_key = os.getenv('QDL_API_KEY')
    api_key_id = os.getenv('QDL_API_KEY_ID')
    
    if not api_key or not api_key_id:
        print("Please set QDL_API_KEY and QDL_API_KEY_ID environment variables")
        print("   Example:")
        print("   export QDL_API_KEY='your-api-key'")
        print("   export QDL_API_KEY_ID='your-api-key-id'")
        return None
    
    # Option 2: Direct assignment (not recommended for production)
    # api_key = "your-api-key"
    # api_key_id = "your-api-key-id"
    
    client = qdl.Client(api_key=api_key, api_key_id=api_key_id)
    print("Quantdle client initialized successfully!")
    return client

# Initialize the client
client = setup_client()


ModuleNotFoundError: No module named 'quantdle'

# Example 1: Basic Data Download

Let's start with downloading basic market data for a single symbol and timeframe.


In [ ]:
# Download 1 month of EURUSD hourly data
print("Downloading EURUSD H1 data for January 2024...")

df = client.download_data(
    symbol="EURUSD",
    timeframe="H1", 
    start_date="2024-01-01",
    end_date="2024-01-31"
)

print(f"Downloaded {len(df):,} data points")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")

# Display sample data
print("\nSample data:")
df.head(10)


In [ ]:
# Basic statistics
print("Basic statistics:")
df[['open', 'high', 'low', 'close', 'volume']].describe()


# Example 2: Multiple Symbols and Timeframes

Download data for multiple symbols and timeframes efficiently.


In [ ]:
# Define the symbols and timeframes we want to download
symbols_config = [
    {"symbol": "EURUSD", "timeframe": "D1", "name": "EUR/USD Daily"},
    {"symbol": "GBPUSD", "timeframe": "H4", "name": "GBP/USD 4-Hour"},
    {"symbol": "XAUUSD", "timeframe": "H1", "name": "Gold 1-Hour"},
]

# Download data for the last 30 days
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")

datasets = {}

for config in symbols_config:
    try:
        print(f"Downloading {config['name']} ({config['symbol']} {config['timeframe']})...")
        
        df = client.download_data(
            symbol=config["symbol"],
            timeframe=config["timeframe"],
            start_date=start_date,
            end_date=end_date,
            show_progress=True
        )
        
        datasets[config["symbol"]] = df
        print(f"Downloaded {len(df):,} data points for {config['name']}")
        
    except Exception as e:
        print(f"Error downloading {config['symbol']}: {e}")

# Compare closing prices
if datasets:
    print(f"\nLatest closing prices:")
    for symbol, df in datasets.items():
        if not df.empty:
            latest = df.iloc[-1]
            print(f"   {symbol}: {latest['close']:.5f} (Volume: {latest['volume']:,})")


# Example 3: Using Polars DataFrames

Demonstrate using polars DataFrames for better performance with large datasets.

> **Note**: Requires polars: `pip install quantdle[polars]`


In [ ]:
try:
    # Check if polars is available
    import polars as pl
    print("Polars is available")
    
    print("Downloading BTCUSD data as Polars DataFrame...")
    
    df_polars = client.download_data(
        symbol="BTCUSD",
        timeframe="H1",
        start_date="2024-01-01",
        end_date="2024-01-15",
        output_format="polars"
    )
    
    print(f"Downloaded {len(df_polars):,} data points as Polars DataFrame")
    print(f"DataFrame info: {df_polars.shape[0]} rows {df_polars.shape[1]} columns")
    
    # Demonstrate some polars operations
    print("\nPolars DataFrame operations:")
    
    # Calculate daily returns
    daily_returns = df_polars.with_columns([
        (pl.col("close").pct_change() * 100).alias("return_pct")
    ])
    
    print("Sample with returns:")
    daily_returns.head(10)
    
except ImportError:
    print("Polars not available. Install with: pip install quantdle[polars]")
    print("Falling back to pandas DataFrame...")
    
    df_polars = client.download_data(
        symbol="XAUUSD",
        timeframe="H1",
        start_date="2024-01-01",
        end_date="2024-01-15",
        output_format="pandas"  # Use pandas instead
    )
    
    print(f"Downloaded {len(df_polars):,} data points as Pandas DataFrame")
    
except Exception as e:
    print(f"Error downloading XAUUSD data: {e}")


In [ ]:
# If we have polars, show some advanced operations
try:
    import polars as pl
    
    # Calculate some statistics using polars
    stats = df_polars.select([
        pl.col("high").max().alias("max_high"),
        pl.col("low").min().alias("min_low"),
        pl.col("volume").mean().alias("avg_volume"),
        pl.col("close").std().alias("price_volatility")
    ])
    
    print("Statistical summary using Polars:")
    stats
    
except ImportError:
    print("Polars operations skipped - polars not available")


# Example 4: Symbol Discovery and Information

Discover available symbols and get information about them.


In [ ]:
# Get all available symbols
print("Discovering available symbols...")
symbols = client.get_available_symbols()

print(f"Found {len(symbols)} available symbols")
print(f"Sample symbols: {symbols[:10]}")

# Get detailed information for some popular symbols
popular_symbols = ['EURUSD', 'GBPUSD', 'XAUUSD', 'BTCUSD']

print("\nSymbol Information:")
print("-" * 80)

symbol_info = {}
for symbol in popular_symbols:
    if symbol in symbols:
        try:
            info = client.get_symbol_info(symbol)
            symbol_info[symbol] = info
            
            print(f"{symbol}:")
            print(f"   Available from: {info.get('available_from', 'N/A')}")
            print(f"   Available to: {info.get('available_to', 'N/A')}")
            print(f"   Data points: {info.get('data_points', 'N/A'):,}")
            print()
            
        except Exception as e:
            print(f"Could not get info for {symbol}: {e}")
    else:
        print(f"{symbol} not available in your account")


# Example 5: Advanced Data Analysis

Perform technical analysis on downloaded data.


In [ ]:
# Use the EURUSD data we downloaded earlier for analysis
analysis_df = df.copy()

# Ensure timestamp is datetime and set as index
if 'timestamp' in analysis_df.columns:
    analysis_df['timestamp'] = pd.to_datetime(analysis_df['timestamp'])
    analysis_df = analysis_df.set_index('timestamp')

print("Technical Analysis Examples:")

# Calculate moving averages
analysis_df['MA_20'] = analysis_df['close'].rolling(window=20).mean()
analysis_df['MA_50'] = analysis_df['close'].rolling(window=50).mean()

# Calculate daily returns
analysis_df['daily_return'] = analysis_df['close'].pct_change()

# Calculate volatility (20-day rolling standard deviation)
analysis_df['volatility'] = analysis_df['daily_return'].rolling(window=20).std() * 100

# RSI calculation (simplified)
def calculate_rsi(prices, window=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

analysis_df['RSI'] = calculate_rsi(analysis_df['close'])

# Display the enhanced dataframe
print("\nData with technical indicators:")
analysis_df[['close', 'MA_20', 'MA_50', 'daily_return', 'volatility', 'RSI']].tail(10)


In [ ]:
# Print latest technical indicators
latest = analysis_df.iloc[-1]
print(f"Latest Technical Indicators:")
print(f"   Price: {latest['close']:.5f}")
print(f"   20-day MA: {latest['MA_20']:.5f}")
print(f"   50-day MA: {latest['MA_50']:.5f}")
print(f"   Daily Return: {latest['daily_return']*100:.2f}%")
print(f"   Volatility (20d): {latest['volatility']:.2f}%")
print(f"   RSI: {latest['RSI']:.1f}")

# Trading signals
print(f"\nSimple Trading Signals:")

if latest['MA_20'] > latest['MA_50']:
    print("   Bullish: 20-day MA above 50-day MA")
else:
    print("   Bearish: 20-day MA below 50-day MA")
    
if latest['RSI'] > 70:
    print("    Overbought: RSI > 70")
elif latest['RSI'] < 30:
    print("    Oversold: RSI < 30")
else:
    print("   Normal: RSI in neutral zone")


In [ ]:
# Statistical summary
print(f"Statistical Summary (last 100 periods):")
recent_data = analysis_df.tail(100)
summary_stats = {
    'Avg Daily Return': f"{recent_data['daily_return'].mean()*100:.3f}%",
    'Volatility': f"{recent_data['daily_return'].std()*100:.2f}%",
    'Max Drawdown': f"{((recent_data['close'] / recent_data['close'].cummax()) - 1).min()*100:.2f}%",
    'Sharpe Ratio': f"{(recent_data['daily_return'].mean() / recent_data['daily_return'].std()) * (252**0.5):.2f}"
}

for metric, value in summary_stats.items():
    print(f"   {metric}: {value}")


# Example 6: Error Handling Best Practices

Demonstrate proper error handling when working with the API.


In [ ]:
# Test with invalid symbol
print("Testing error handling with invalid symbol...")
try:
    df = client.download_data(
        symbol="INVALID_SYMBOL",
        timeframe="H1",
        start_date="2024-01-01",
        end_date="2024-01-02"
    )
    print("This should have failed!")
except Exception as e:
    print(f"Properly caught error: {e}")

# Test with invalid date range
print("\nTesting with invalid date range...")
try:
    df = client.download_data(
        symbol="EURUSD",
        timeframe="H1", 
        start_date="2025-01-01",  # Future date
        end_date="2025-01-02"
    )
    print("This should have failed!")
except Exception as e:
    print(f"Properly caught error: {e}")


In [ ]:
# Robust download function with retry logic
def robust_download(client, symbol, timeframe, start_date, end_date, max_retries=3):
    """Download data with retry logic and proper error handling."""
    for attempt in range(max_retries):
        try:
            print(f"   Attempt {attempt + 1}/{max_retries} for {symbol}...")
            df = client.download_data(
                symbol=symbol,
                timeframe=timeframe,
                start_date=start_date,
                end_date=end_date,
                show_progress=False
            )
            print(f"   Success! Downloaded {len(df):,} records")
            return df
            
        except Exception as e:
            print(f"   Attempt {attempt + 1} failed: {e}")
            if attempt == max_retries - 1:
                print(f"   All {max_retries} attempts failed for {symbol}")
                return None
            else:
                print(f"   Retrying in 1 second...")
                import time
                time.sleep(1)
    
    return None

# Test the robust function
print("\n Testing robust download function:")
df_robust = robust_download(client, "EURUSD", "H1", "2024-01-01", "2024-01-02")


# Example 7: Large Date Range with Chunking

Download data for a large date range, demonstrating automatic chunking for better performance.


In [ ]:
# Download a larger dataset to demonstrate chunking
print("Downloading 1 year of EURUSD daily data...")
print("   (This demonstrates automatic chunking for large requests)")

try:
    large_df = client.download_data(
        symbol="EURUSD",
        timeframe="D1",
        start_date="2023-01-01", 
        end_date="2023-12-31",
        max_workers=6,           # Increase parallel downloads
        show_progress=True,      # Show progress bars
        chunk_size_years=1       # Use 1-year chunks
    )
    
    print(f"Downloaded {len(large_df):,} daily candles")
    print(f"Date range: {large_df['timestamp'].min()} to {large_df['timestamp'].max()}")
    
    # Show sample of the data
    print("\nSample of downloaded data:")
    large_df.head()
    
except Exception as e:
    print(f"Error downloading large dataset: {e}")
    large_df = None


In [ ]:
# Analyze the large dataset if we have it
if large_df is not None and not large_df.empty:
    # Analyze the data by month
    print("Monthly summary:")
    large_df['timestamp'] = pd.to_datetime(large_df['timestamp'])
    large_df['month'] = large_df['timestamp'].dt.to_period('M')
    
    monthly_stats = large_df.groupby('month').agg({
        'high': 'max',
        'low': 'min', 
        'close': ['first', 'last'],
        'volume': 'mean'
    }).round(5)
    
    # Show first few months
    print("First 6 months of 2023:")
    monthly_stats.head(6)
else:
    print(" No large dataset available for analysis")


# Summary

## 🎉 Congratulations!

You've successfully explored the key features of the Quantdle Python client:

✅ **Basic data downloads** - Simple retrieval of historical market data  
✅ **Multiple symbols** - Efficient batch processing of different assets  
✅ **Data formats** - Working with both pandas and polars DataFrames  
✅ **Symbol discovery** - Finding available data and getting metadata  
✅ **Technical analysis** - Real-world data analysis and indicators  
✅ **Error handling** - Robust code for production environments  
✅ **Large datasets** - Automatic chunking for better performance  

## Next Steps

1. **Explore your data**: Try downloading data for symbols available in your account
2. **Build strategies**: Use the technical indicators as a starting point for trading strategies
3. **Optimize performance**: Experiment with different `max_workers` and `chunk_size_years` settings
4. **Production deployment**: Implement proper error handling and logging for production use

## Resources

- 📚 [Quantdle Documentation](https://docs.quantdle.com)
- 🐛 [GitHub Issues](https://github.com/quantdle/quantdle-python/issues)
- 💬 [Community Support](https://quantdle.com)

Happy trading! 📈
